In [ ]:
# Create a virtual environment and install dependencies
#!python -m venv venv

# Activate the virtual environment
! .\env\Scripts\Activate

In [ ]:
!pip install pandas python-dateutil
!pip install kafka-python


In [1]:
import os, json, pandas as pd
from dateutil import tz
from dateutil import parser as dateparser
from datetime import datetime

In [ ]:
df = pd.read_csv('../dataset/Camera_Traffic_Counts_20251108.csv')
df.head()

In [ ]:
# Clean column names: strip whitespace, convert to lowercase, replace spaces with underscores
df.columns = (df.columns
              .str.strip()
              .str.lower()
              .str.replace(r"\s+", "_", regex=True))

print("\n Clean Columns:")
print('-----------------------------------')
print(df.columns)

print("\n Data:")
print('-----------------------------------')
#df.head()
 


 Clean Columns:
-----------------------------------
Index(['record_id', 'atd_device_id', 'read_date', 'intersection_name',
       'direction', 'movement', 'heavy_vehicle', 'volume',
       'speed_average_(miles_per_hour)', 'speed_stddev',
       'seconds_in_zone_average', 'seconds_in_zone_stddev', 'month', 'day',
       'year', 'hour', 'minute', 'day_of_week', 'bin_duration_(seconds)'],
      dtype='object')

 Data:
-----------------------------------


,record_id,atd_device_id,read_date,intersection_name,direction,movement,heavy_vehicle,volume,speed_average_(miles_per_hour),speed_stddev,seconds_in_zone_average,seconds_in_zone_stddev,month,day,year,hour,minute,day_of_week,bin_duration_(seconds)
0,807feb986678a9d1502500e45b95cbfb,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,LEFT TURN,False,2.0,29.500,37.477,0.9,1.131,11.0,15.0,"2,020",8.0,30.0,0.0,900.0
1,b68106cfd81e9616f0d212594a40fe4d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,RIGHT TURN,False,1.0,9.000,0.000,6,0,11.0,15.0,"2,020",8.0,30.0,0.0,900.0
2,264064fe0b8c1efa75746379aa8fa26d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,False,98.0,31.265,9.358,4.347,3.614,11.0,15.0,"2,020",8.0,30.0,0.0,900.0
3,73259de28a6c14f9b8a5fac147197530,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,True,2.0,26.500,13.435,5.5,3.536,11.0,15.0,"2,020",8.0,30.0,0.0,900.0
4,b3e53e069b1b2dc39b0499e0d8f01e79,"6,547",2020 Nov 15 08:45:00 AM,RIVERSIDE DR / CROSSING PL,EASTBOUND,LEFT TURN,False,22.0,14.227,5.968,5.073,7.688,11.0,15.0,"2,020",8.0,45.0,0.0,900.0


In [26]:
print("\nData Shape:")
print('-----------------------------------')
print(df.shape)

print("\nData Types:")
print('-----------------------------------')
print(df.dtypes)

print("\nInfo:")
print('-----------------------------------')
print(df.info())


Data Shape:
-----------------------------------
(98175, 19)

Data Types:
-----------------------------------
record_id                          object
atd_device_id                      object
read_date                          object
intersection_name                  object
direction                          object
movement                           object
heavy_vehicle                      object
volume                            float64
speed_average_(miles_per_hour)    float64
speed_stddev                      float64
seconds_in_zone_average            object
seconds_in_zone_stddev             object
month                             float64
day                               float64
year                               object
hour                              float64
minute                            float64
day_of_week                       float64
bin_duration_(seconds)            float64
dtype: object

Info:
-----------------------------------
<class 'pandas.core.frame.DataFrame

In [ ]:
df['datetime'] = pd.to_datetime(df['read_date'],
                                errors='coerce')

# localize to America/Chicago, then convert to UTC
s = pd.to_datetime(df['read_date'], errors='coerce')
s = (s
     .dt.tz_localize('America/Chicago', nonexistent='shift_forward', ambiguous='NaT')
     .dt.tz_convert('UTC'))

df['datetime_utc'] = s
df['timestamp_ms'] = (s.view('int64') // 10**6).astype('Int64')  # tz-aware -> use .view('int64')
                                                     
df.head()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_35732\3677952020.py:11: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  df['timestamp_ms'] = (s.view('int64') // 10**6).astype('Int64')  # tz-aware -> use .view('int64')


,record_id,atd_device_id,read_date,intersection_name,direction,movement,heavy_vehicle,volume,speed_average_(miles_per_hour),speed_stddev,...,month,day,year,hour,minute,day_of_week,bin_duration_(seconds),datetime,datetime_utc,timestamp_ms
0,807feb986678a9d1502500e45b95cbfb,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,LEFT TURN,False,2.0,29.500,37.477,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
1,b68106cfd81e9616f0d212594a40fe4d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,RIGHT TURN,False,1.0,9.000,0.000,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
2,264064fe0b8c1efa75746379aa8fa26d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,False,98.0,31.265,9.358,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
3,73259de28a6c14f9b8a5fac147197530,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,True,2.0,26.500,13.435,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
4,b3e53e069b1b2dc39b0499e0d8f01e79,"6,547",2020 Nov 15 08:45:00 AM,RIVERSIDE DR / CROSSING PL,EASTBOUND,LEFT TURN,False,22.0,14.227,5.968,...,11.0,15.0,2020.0,8.0,45.0,0.0,900.0,2020-11-15 08:45:00,2020-11-15 14:45:00+00:00,1605451500000


In [44]:
print("\nData Types:")
print('-----------------------------------')
print(df.dtypes)

df.head()


Data Types:
-----------------------------------
record_id                                      object
atd_device_id                                  object
read_date                                      object
intersection_name                              object
direction                                      object
movement                                       object
heavy_vehicle                                  object
volume                                        float64
speed_average_(miles_per_hour)                float64
speed_stddev                                  float64
seconds_in_zone_average                        object
seconds_in_zone_stddev                         object
month                                         float64
day                                           float64
year                                          float64
hour                                          float64
minute                                        float64
day_of_week                      

,record_id,atd_device_id,read_date,intersection_name,direction,movement,heavy_vehicle,volume,speed_average_(miles_per_hour),speed_stddev,...,month,day,year,hour,minute,day_of_week,bin_duration_(seconds),datetime,datetime_utc,timestamp_ms
0,807feb986678a9d1502500e45b95cbfb,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,LEFT TURN,False,2.0,29.500,37.477,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
1,b68106cfd81e9616f0d212594a40fe4d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,RIGHT TURN,False,1.0,9.000,0.000,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
2,264064fe0b8c1efa75746379aa8fa26d,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,False,98.0,31.265,9.358,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
3,73259de28a6c14f9b8a5fac147197530,"6,547",2020 Nov 15 08:30:00 AM,RIVERSIDE DR / CROSSING PL,WESTBOUND,THRU,True,2.0,26.500,13.435,...,11.0,15.0,2020.0,8.0,30.0,0.0,900.0,2020-11-15 08:30:00,2020-11-15 14:30:00+00:00,1605450600000
4,b3e53e069b1b2dc39b0499e0d8f01e79,"6,547",2020 Nov 15 08:45:00 AM,RIVERSIDE DR / CROSSING PL,EASTBOUND,LEFT TURN,False,22.0,14.227,5.968,...,11.0,15.0,2020.0,8.0,45.0,0.0,900.0,2020-11-15 08:45:00,2020-11-15 14:45:00+00:00,1605451500000


In [ ]:
# Check if any record has null values
df_has_null = df.isnull().any().any()

# List of columns with null values
columns_with_null = df.columns[df.isnull().any()].tolist()

print(df_has_null)           # True if any nulls exist
print(columns_with_null)     # Columns that contain null values

True
['read_date', 'intersection_name', 'direction', 'movement', 'heavy_vehicle', 'volume', 'speed_average_(miles_per_hour)', 'speed_stddev', 'seconds_in_zone_average', 'seconds_in_zone_stddev', 'month', 'day', 'year', 'hour', 'minute', 'day_of_week', 'bin_duration_(seconds)', 'datetime', 'datetime_utc']


In [ ]:
# Show missing value counts based on the column names
print("\n Missing value summary (exact string '',NaN):")
missing_report = pd.DataFrame({
    'missing_count_na': df.isna().sum(),
    'empty_count_na': (df == '').sum(),
    'unique_values': df.nunique(dropna=False)
}).sort_values('missing_count_na', ascending=False)
display(missing_report.head(30))


 Missing value summary (exact string '',NaN):


,missing_count_na,empty_count_na,unique_values
intersection_name,1,0,45
read_date,1,0,389
direction,1,0,6
movement,1,0,5
volume,1,0,418
heavy_vehicle,1,0,3
year,1,0,3
hour,1,0,25
speed_average_(miles_per_hour),1,0,17158
speed_stddev,1,0,17551


In [48]:
# Drop rows where ANY value is missing
df_main_rows = df.dropna()